In [166]:
import pandas as pd
import os

In [167]:
data_path = r"C:\Projects\connecteddatahub\data\cleaned_dataframes"
years = ["1999","2000","2005","2007","2008","2009","2010","2011","2013","2018",]
# years = ['1999']
data_path = r"C:\Projects\connecteddatahub\data\cleaned_dataframes"


In [168]:
# # collect unique institutions per year
# unique_institutions = {}
# for year in years:
#     board_df = pd.read_csv(os.path.join(data_path, 'boards', f'{year}_boards.csv'))
#     board_df = board_df[board_df['PrimarySample'] == True]

#     unique_institutions[year] = set(board_df['Institution'].dropna().str.strip().unique())

# # build the full sorted list of institutions
# all_insts = sorted(set().union(*unique_institutions.values()))

# # split into those appearing every year and those missing some
# complete_insts   = []
# incomplete_insts = []

# for inst in all_insts:
#     missing = [year for year in years if inst not in unique_institutions[year]]
#     if not missing:
#         complete_insts.append(inst)
#     else:
#         incomplete_insts.append((inst, missing))

# # print institutions present in every year
# for inst in complete_insts:
#     print(f"{inst}: appears in all years")

# # print the rest with their missing years
# for inst, missing in incomplete_insts:
#     present = [str(year) for year in years if inst in unique_institutions[year]]
#     missing_str  = ", ".join(str(y) for y in missing)
#     present_str  = ", ".join(present)
#     print(f"{inst}: appears in {present_str}; missing {missing_str}")


In [169]:
# Collect per-year data and build maps
unique_institutions = {}
per_year_sid       = {}
inst_to_sid        = {}

for year in years:
    board_df = pd.read_csv(os.path.join(data_path, 'boards', f'{year}_boards.csv'))
    board_df = board_df[(board_df['PrimarySample'] == True) | board_df['SystemId'].notna()]

    # institution → SystemId for this year
    sid_map = board_df.set_index('Institution')['SystemId'].to_dict()
    per_year_sid[year] = sid_map

    # record unique institution names
    unique_institutions[year] = set(sid_map.keys())

    # populate global inst→SystemId (first-seen)
    for inst, sid in sid_map.items():
        if inst not in inst_to_sid:
            inst_to_sid[inst] = sid

# Find all institutions and which years they’re missing
all_insts   = sorted(inst_to_sid.keys())
incomplete  = []
for inst in all_insts:
    missing_years = [y for y in years if inst not in unique_institutions[y]]
    if missing_years:
        incomplete.append((inst, missing_years))

# Print out results with plain print
print("Institutions missing in some years and their peers sharing SystemId:\n")
for inst, missing_years in incomplete:
    sid = inst_to_sid[inst]
    missing_str = ", ".join(str(y) for y in missing_years)
    print(f"{inst} (SystemId={sid}) is missing in: {missing_str}")
    for y in missing_years:
        peers = [
            other for other, other_sid in per_year_sid[y].items()
            if other_sid == sid and other != inst
        ]
        if peers:
            peer_list = ", ".join(sorted(peers))
            print(f"  Year {y} peers: {peer_list}")
        else:
            print(f"  Year {y} peers: (none)")
    print()  # blank line between institutions


Institutions missing in some years and their peers sharing SystemId:

Arizona Board Of Regents (SystemId=1000.0) is missing in: 1999, 2000, 2005, 2008, 2009
  Year 1999 peers: Arizona State University, Northern Arizona University, University Of Arizona
  Year 2000 peers: Arizona State University, Northern Arizona University, University Of Arizona
  Year 2005 peers: Arizona State University, Northern Arizona University, University Of Arizona
  Year 2008 peers: Arizona State University, Northern Arizona University, University Of Arizona
  Year 2009 peers: Arizona State University, Northern Arizona University, University Of Arizona

Arizona State University (SystemId=1000.0) is missing in: 2007, 2010, 2011, 2013, 2018
  Year 2007 peers: Arizona Board Of Regents, Northern Arizona University, University Of Arizona
  Year 2010 peers: Arizona Board Of Regents
  Year 2011 peers: Arizona Board Of Regents
  Year 2013 peers: Arizona Board Of Regents
  Year 2018 peers: Arizona Board Of Regents

Ba

In [170]:
# Collect per‑year data and build maps
unique_institutions = {}
per_year_sid       = {}
inst_to_sid        = {}

for year in years:
    board_df = pd.read_csv(
        os.path.join(data_path, 'boards', f'{year}_boards.csv')
    )
    board_df = board_df[
        (board_df['PrimarySample'] == True) |
        board_df['SystemId'].notna()
    ]

    sid_map = board_df.set_index('Institution')['SystemId'].to_dict()
    per_year_sid[year]        = sid_map
    unique_institutions[year] = set(sid_map.keys())

    for inst, sid in sid_map.items():
        inst_to_sid.setdefault(inst, sid)

# Now, for each year, find institutions that are missing AND have no peers
print("Missing institutions by year (only those without any same‑SystemId peers):\n")
for year in years:
    lonely_insts = []
    peer_info    = {}
    for inst, sid in inst_to_sid.items():
        if inst in unique_institutions[year]:
            continue  # present this year, skip

        # find peers in this year
        peers = [
            other for other, other_sid in per_year_sid[year].items()
            if other_sid == sid and other != inst
        ]
        if not peers:
            lonely_insts.append((inst, sid))
        else:
            peer_info[inst] = peers

    # print the header for this year
    print(f"Year {year}:")
    if lonely_insts:
        for inst, sid in sorted(lonely_insts):
            print(f"  {inst} (SystemId={sid}) — no peers")
    else:
        print("  All missing institutions have peers")

    # if you also want to show which missing ones did have peers:
    if peer_info:
        print("  (Missing but with peers:)")
        for inst, peers in sorted(peer_info.items()):
            print(f"    {inst} → peers: {', '.join(sorted(peers))}")
    print()  # blank line between years


Missing institutions by year (only those without any same‑SystemId peers):

Year 1999:
  Baylor College Of Medicine (SystemId=nan) — no peers
  Colorado State University System (SystemId=4210131357.0) — no peers
  Indiana University System (SystemId=1001.0) — no peers
  New School (SystemId=nan) — no peers
  Northeastern University (SystemId=nan) — no peers
  Northwestern University (SystemId=nan) — no peers
  Oregon Health & Science University (SystemId=nan) — no peers
  Polytechnic Institute Of New York University (SystemId=nan) — no peers
  Rockefeller University (SystemId=nan) — no peers
  Rutgers State University Of New Jersey (SystemId=nan) — no peers
  Saint Louis University (SystemId=nan) — no peers
  Southern Illinois University System (SystemId=nan) — no peers
  Southern Methodist University (SystemId=nan) — no peers
  Stevens Institute Of Technology (SystemId=nan) — no peers
  Union Institute & University (SystemId=nan) — no peers
  University Of Pennsylvania (SystemId=nan) 

In [171]:
# unique_institutions = {}
# per_year_sid = {}
# inst_to_sid = {}

# for year in years:
#     board_csv = os.path.join(data_path, 'boards', f'{year}_boards.csv')
#     board_df = pd.read_csv(board_csv)
#     board_df = board_df[(board_df['PrimarySample'] == True) | board_df['SystemId'].notna()]

#     sid_map = board_df.set_index('Institution')['SystemId'].to_dict()
#     per_year_sid[year] = sid_map
#     unique_institutions[year] = set(sid_map.keys())

#     for inst, sid in sid_map.items():
#         if inst not in inst_to_sid:
#             inst_to_sid[inst] = sid

# for year in sorted(years):
#     # for each known institution, check if it is missing and has no peers
#     missing = []
#     for inst, sid in inst_to_sid.items():
#         if inst in unique_institutions[year]:
#             continue
#         peers = [
#             other for other, other_sid in per_year_sid[year].items()
#             if other_sid == sid and other != inst
#         ]
#         if not peers:
#             missing.append((inst, sid))

#     if not missing:
#         continue

#     print(f"Year {year}:")
#     for inst, sid in sorted(missing):
#         print(f"  {inst} (SystemId={sid}) is missing and has no peers")
#     print()

In [172]:
unique_institutions = {}
per_year_sid        = {}
inst_to_sid         = {}
inst_to_affil       = {}

for year in years:
    board_csv = os.path.join(data_path, 'boards', f'{year}_boards.csv')
    board_df  = pd.read_csv(board_csv)
    board_df  = board_df[(board_df['PrimarySample'] == True)
                        | board_df['SystemId'].notna()]

    sid_map   = board_df.set_index('Institution')['SystemId'].to_dict()
    affil_map = board_df.set_index('Institution')['AffiliationId'].to_dict()

    per_year_sid[year]       = sid_map
    unique_institutions[year] = set(sid_map.keys())

    for inst, sid in sid_map.items():
        if inst not in inst_to_sid:
            inst_to_sid[inst] = sid

    for inst, affil in affil_map.items():
        if inst not in inst_to_affil and pd.notna(affil):
            inst_to_affil[inst] = affil

print(inst_to_affil)

for year in sorted(years):
    missing = []
    sid_map = per_year_sid[year]

    for inst, sid in inst_to_sid.items():
        if inst in unique_institutions[year]:
            continue

        # check for peers by SystemId
        peers = [
            other for other, other_sid in sid_map.items()
            if other_sid == sid and other != inst
        ]
        if peers:
            continue

        # backup check by global AffiliationId
        affil = inst_to_affil.get(inst)
        if affil is not None:
            same_affil = [
                other for other in sid_map
                if inst_to_affil.get(other) == affil and other != inst
            ]
            if same_affil:
                continue

        missing.append((inst, sid))

    if not missing:
        continue

    print(f"Year {year}:")
    for inst, sid in sorted(missing):
        print(f"  {inst} (SystemId={sid}) is missing without peers or affiliation backup")
    print()


{'Adelphi University': 71965598.0, 'American University': 181401687.0, 'Andrews University': 102298084.0, 'Arizona State University': 55732556.0, 'Auburn University': 82497590.0, 'Ball State University': 198089087.0, 'Baylor University': 157394403.0, 'Biola University': 152479009.0, 'Boston College': 103531236.0, 'Boston University': 111088046.0, 'Bowling Green State University': 157417397.0, 'Brandeis University': 6902469.0, 'Brigham Young University': 100005738.0, 'Brown University': 27804330.0, 'California Institute Of Technology': 122411786.0, 'California State University System': 127339247.0, 'Carnegie Mellon University': 74973139.0, 'Case Western Reserve University': 58956616.0, 'Catholic University Of America': 84470341.0, 'Central Michigan University': 1629065.0, 'Chapman University': 185071736.0, 'City University Of New York': 174216632.0, 'Clark Atlanta University': 52064589.0, 'Clark University': 130785548.0, 'Clarkson University': 16944753.0, 'Clemson University': 8078737.0